Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Setup AML Workspace
---

This notebook walks you through all the necessary steps to configure your environment for this solution accelerator including:

1. Setting the path to the utils folder to take advantage of environment variables
2. Connecting to your workspace 
3. Create a config.json (this can be skipped if running on an AML compute instance)
4. Create a compute cluster


## Environment Variable

This will allow the user or operations to set environment variables on the local compute or the build agent. This way values can be changed simply by using the name of the environment variable and replacing it with a new value whether that occurs locally or while deploying through Operations.

[NOTE] If you change values in the .env file you must close and re-open your IDE. This is necessary because the values in the .env file are set on open. Similarly, if you change values on the compute you will have to restart terminal. If you are using Linux or Mac you can restart your bash by executing `source ~/.bashrc`.


In [ ]:
import sys
sys.path.append("../../")

from utils.env_variables import Env
from utils.aml_workspace import Connect
e=Env()

## 1.0 Connect to workspace

Connect this solution accelerator to your AML workspace. This step isn't necessary if you're using a Notebook VM.

The following cell allows you to specify your workspace parameters. This cell uses the python method os.getenv to read values from environment variables which is useful for automation. If no environment variable exists, the parameters will be set to the specified default values.

In [ ]:
import os

subscription_id = e.subscription_id
resource_group = e.resource_group
workspace_name = e.workspace_name
workspace_region = e.workspace_region

In [ ]:
from azureml.core import Workspace

try:
    connect = Connect()
    ws = connect.authenticate()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace does not exist. Creating workspace")
    ws = Workspace.create(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group,
                            location=workspace_region, create_resource_group=True, sku='enterprise', exist_ok=True)

In [ ]:
ws.get_details()

## 2.0 Write config file
Write the details of the workspace to a config.json file:

In [1]:
from azureml.core import Workspace
ws = Workspace.from_config()
ws.write_config()

## 3.0 Create compute cluster

You will need a compute cluster for training and batch forecasting.
This is a one-time set up so you won't need to re-run this in future notebooks, but you'll need to use the same cluster name to reference this cluster in following notebooks.

In [2]:
# Choose a name for your CPU cluster
cpu_cluster_name = e.compute_name

We create a STANDARD_D13_V2 compute cluster. D-series VMs are used for tasks that require higher compute power and temporary disk performance. This [page](https://docs.microsoft.com/azure/cloud-services/cloud-services-sizes-specs) will give you more information on VM sizes to help you decide which will best fit your use case.

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found an existing cluster, using it instead.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=e.vm_size,
                                                           min_nodes=e.min_nodes,
                                                           max_nodes=e.max_nodes)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)

InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Next Steps

Now that the AML Workspace its configured, it's time to create the sample datasets. Follow the steps in [01_Data_Preparation.ipynb](01_Data_Preparation.ipynb) for that.